In [ ]:
import pandas as pd
import numpy as np
from fitparse import FitFile

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
def get_session_time(file_path):
    """A function to return the `time_created` field from a FIT file
    
    Parameters
    ----------
    file_path : str
        The file location of the FIT file

    Returns
    -------
    datetime
        <class 'datetime.datetime'> representation of when the FIT file was created on
    """
    with FitFile(open(file_path, 'rb')) as fitfile:
        fitfile._parse_message() # parse first message as header
        return fitfile._parse_message().get_value('time_created') # parse second message and look for `time_created`
    return None

In [ ]:
def get_sessions():
    """A function to return a list of FIT sessions in the data directory with its creation time.

    Returns
    -------
    list
        A list of tuples with the file path as first element and its creation time as the second element.
    """
    datafiles = !ls data/*.fit
    sessions = []
    for filepath in datafiles:
        sessions.append((filepath,get_session_time(filepath)))
    return sessions

In [ ]:
def load_session_data(file_path):
    """A function to return the `time_created` field from a FIT file
    
    Some columns are dropped and renamed. Only keep rows where heart_rate data exists.
    Resample to 1 second frequency with interpolated values.
    
    Parameters
    ----------
    file_path : str
        The file location of the FIT file

    Returns
    -------
    dataframe
        Pandas DataFrame of the FIT file
    """
    with FitFile(open(file_path, 'rb')) as fitfile:
        df = pd.DataFrame([record.get_values() for record in fitfile.get_messages('record')])
        df.drop(columns=['unknown_87','unknown_88','unknown_90','altitude','fractional_cadence','speed'], inplace=True)
        df.rename(columns={"enhanced_speed": "speed", "enhanced_altitude": "altitude"}, inplace=True)
        df = df.dropna(subset=['heart_rate'])
        df.set_index('timestamp', inplace=True)
        df = df.resample('s').interpolate()
        return df    

In [ ]:
sessions = get_sessions()
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Heart Rate Dashboard"),
    html.Label([
        "Workout Sessions",
        dcc.Dropdown(
            id='sessions-dropdown', clearable=False,
            value = sessions[0][0],
            options=[
                {'label': d, 'value': f}
                for f,d in sessions
            ])
    ]),
    html.Div(id='output_text', children="HI"),
])
# Define callback to update graph
@app.callback(
    Output('output_text', 'children'),
    [Input("sessions-dropdown", "value")]
)
def change_session(session):
    df = load_session_data(session)
    return "New session selected is {}<strong>{}</strong>".format(session,df)

# Run app and display result inline in the notebook
app.run_server(mode='external')

In [ ]:
for s in get_sessions():
    df = load_session_data(s[0])
    display(df)
    break